# Speed Comparison

In [1]:
from datetime import timedelta
from math import log, exp

from boundedexp import MeasurementIsotope

In [2]:
measurement_time: timedelta = timedelta(seconds=60000)
half_life: timedelta = timedelta(hours=9.14)

In [3]:
mean_half_life = half_life / log(2)

In [4]:
num_samples = 1_000_000

## Rust/maturin library

In [5]:
mi = MeasurementIsotope(half_life=half_life, measurement_time=measurement_time)

### Calling one at a time

In [6]:
%%timeit
[mi.get_time() for _ in range(num_samples)]

254 ms ± 2.29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Using rust/rayon to parallelize the calls

In [7]:
%%timeit
_ = mi.get_times(num_samples)

185 ms ± 4.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Pure python

In [8]:
from random import random

In [9]:
def inv_scale(half_life, measurement_time) -> float:
    mhl: timedelta = half_life / log(2)
    ep: float = -(measurement_time / mhl)
    return 1 - exp(ep)

In [10]:
def inv_cdf(cdf: float, half_life: timedelta, measurement_time: timedelta) -> timedelta:
    """Given a cumulative distribution value, return the equivalent time"""
    mhl: timedelta = half_life / log(2)
    scale: float = inv_scale(half_life, measurement_time)
    return -mhl * log(1 - cdf * scale)

In [11]:
%%timeit
[inv_cdf(random(), half_life, measurement_time) for _ in range(num_samples)]

1.11 s ± 3.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Numpy

In [12]:
import numpy as np

In [13]:
hltus: int = int(half_life.total_seconds() * 1e6)
np_half_life: np.timedelta64 = np.timedelta64(hltus, 'us')
np_measurement_time: np.timedelta64 = np.timedelta64(int(measurement_time.total_seconds()), 's')

In [14]:
rng: np.random.Generator = np.random.default_rng()

In [15]:
def np_inv_cdf(
    num_samples: int, 
    half_life: np.timedelta64, 
    measurement_time: np.timedelta64,
    rng: np.random.Generator
) -> np.ndarray:
    mhl: np.timedelta64 = half_life / np.log(2)
    inv_scale: float = 1 - np.exp(-(measurement_time / mhl))
    return -mhl * np.log(1 - rng.random(size=num_samples) * inv_scale)

In [16]:
%%timeit
np_inv_cdf(num_samples, np_half_life, np_measurement_time, rng)

10.2 ms ± 477 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Numba

In [17]:
from numba import jit

In [ ]:
@jit
def numba_inv_scale(half_life: timedelta, measurement_time: timedelta) -> float:
    mhl: timedelta = half_life / log(2)
    ep: float = -(measurement_time / mhl)
    return 1 - exp(ep)

In [25]:
@jit
def numba_inv_cdf(cdf: float, half_life: timedelta, measurement_time: timedelta) -> timedelta:
    """Given a cumulative distribution value, return the equivalent time"""
    mhl: timedelta = half_life / log(2)
    scale: float = inv_scale(half_life, measurement_time)
    return -mhl * log(1 - cdf * scale)

In [26]:
numba_inv_cdf(random(), half_life, measurement_time)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'inv_scale': Cannot determine Numba type of <class 'function'>

File "../../../../../var/folders/48/l5nlrfb93zj4jy_wh_bx8s8h0000gn/T/ipykernel_2462/3850974732.py", line 5:
<source missing, REPL/exec in use?>

During: Pass nopython_type_inference 

This error may have been caused by the following argument(s):
- argument 1: Cannot determine Numba type of <class 'datetime.timedelta'>
- argument 2: Cannot determine Numba type of <class 'datetime.timedelta'>
